In [2]:
# import the required libraries
import os
import re
import csv
import pandas as pd
import numpy as np
import pyodbc
import shutil

In [15]:
# Set variables for the project (i.e. the input location of the file to be processed and the output location) )

folderlink = '..//data//'
output_txt = folderlink+'output.txt'
output_csv = folderlink+'output_file.csv'
folder_output = 'output//'

In [4]:
# Panda settings for showing data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [5]:
# import the geonames file for the Netherlands (which was obtained here: https://www.geonames.org/ )
gf= pd.read_csv('geonl_sel.csv')

In [13]:
childs = pd.read_csv(folderlink+folder_output+'minister_info.csv' , sep=';', encoding='utf-8')
birth_org = pd.read_csv(folderlink+folder_output+'birth_info.csv', sep=';', encoding='utf-8')
bapt_org = pd.read_csv(folderlink+folder_output+'bapt_info.csv', sep=';', encoding='utf-8') 
death_org = pd.read_csv(folderlink+folder_output+'death_info.csv', sep=';', encoding='utf-8')
df = pd.read_csv(folderlink+folder_output+'parent_data.csv', sep=';', encoding='utf-8')

In [6]:
def export_access_to_dataframes(database_path):
    # Connection string for Access database
    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        r'DBQ=' + database_path + ';'
    )

    # Establish a connection to the Access database
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Get a list of all tables in the database
    tables = [row.table_name for row in cursor.tables(tableType='TABLE')]

    # Loop through the tables and load each into a DataFrame
    for table in tables:
        query = f'SELECT * FROM [{table}]'
        df = pd.read_sql(query, conn)
        globals()[f'tbl_{table}'] = df  # Create a global variable with the table name

    # Close the connection
    conn.close()

In [7]:
# Link with CLERUS database (which is the result of the data Harmonization steps)
clerus_database_path = 'E:\\digidure\\CLERUS_v3_06082024.accdb'
export_access_to_dataframes(clerus_database_path)

C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_13032\1595437475.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_13032\1595437475.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_13032\1595437475.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_13032

In [9]:
role_place = tbl_12_clerus_role.merge(gf, left_on=tbl_12_clerus_role["role_place"].str.extract(f"({'|'.join(gf['placename'])})")[0], right_on="placename", how="inner")

In [16]:
role_place.to_csv(folderlink+folder_output+'role_place_geo.csv', sep=';', encoding='utf-8', index=False)

In [11]:
sorted_gf = gf.sort_values(by='placename', key=lambda x: x.str.len(), ascending=False)
sorted_gf = sorted_gf.reset_index(drop=True)

In [17]:
birth_geo = tbl_01_clerus_bio.merge(sorted_gf, left_on=tbl_01_clerus_bio["birth_place"].str.extract(f"({'|'.join(sorted_gf['placename'])})")[0], right_on="placename", how="left")
birth_geo.to_csv(folderlink+folder_output+'bio_birth_info_geo.csv', sep=';', encoding='utf-8', index=False)

In [18]:
death_geo = tbl_01_clerus_bio.merge(sorted_gf, left_on=tbl_01_clerus_bio["death_place"].str.extract(f"({'|'.join(sorted_gf['placename'])})")[0], right_on="placename", how="left")
death_geo.to_csv(folderlink+folder_output+'death_info_geo.csv', sep=';', encoding='utf-8', index=False)

In [19]:
bapt_geo = tbl_01_clerus_bio.merge(sorted_gf, left_on=tbl_01_clerus_bio["baptism_place"].str.extract(f"({'|'.join(sorted_gf['placename'])})")[0], right_on="placename", how="left")
bapt_geo.to_csv(folderlink+folder_output+'bapt_info_geo.csv', sep=';', encoding='utf-8', index=False)